In [43]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [45]:
import pandas as pd
import pathlib
import sys
df = pd.read_csv("/Users/godsentizinyon/Downloads/Recommender Systems in Python/recommender/spotify_millsongdata.csv")

In [46]:
df.isnull().sum()
df = df.sample(5000).drop("link", axis=1).reset_index(drop=True)

In [47]:
df["text"][1]

"I was dancing when I was twelve  \r\nI was dancing when I was twelve  \r\nI was dancing when I was...oh  \r\nI was dancing when I was...oh  \r\nI danced myself out of the womb  \r\nI danced myself out of the womb  \r\nIs it strange to dance so soon?  \r\nIs it strange to dance so soon?  \r\nIs it wrong to understand  \r\nThe fear that dwells inside a man?  \r\nWhat's it like to be a loon?  \r\nI liken it to a balloon  \r\nOh...  \r\nI danced myself into the tomb  \r\nI danced myself into the tomb  \r\nIs it strange to dance so soon?  \r\nIs it strange to dance so soon?  \r\nIs it wrong to understand  \r\nThe fear that dwells inside a man?  \r\nWhat's it like to be a loon?  \r\nI liken it to a balloon  \r\nOh...\r\n\r\n"

In [48]:
df.shape

(5000, 3)

Text Cleaning/Processing

df['text'].str.lower().replace(r"^\W\s", " ").replace(r"\n", " ", regex = True)

In [49]:
df['text'] = df['text'].str.lower().replace(r"^\W\s", " ").replace(r"\n", " ", regex = True)


TOKENIZATION

df.tail(5)

In [50]:
df.tail(5)

,artist,song,text
4995,XTC,Ra Ra For Red Rocking Horse,did you think you knew the game? \r did you t...
4996,Hanson,Thinking Bout Somethin,"well i gave you love, you know it \r so when ..."
4997,Santana,One Chain (Don't Make No Prison),just because in a moment of madness \r i took...
4998,Jackson Browne,Anything Can Happen,time will come when we know what happened here...
4999,Kelly Clarkson,Empty Handed,when did you know that nothing was right \r w...


In [51]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize


In [52]:
stemmer = PorterStemmer()

In [53]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Download if not already present
nltk.download('punkt')

# Initialize your stemmer
stemmer = PorterStemmer()



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/godsentizinyon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [54]:
import re
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def token(txt):
    # Find words (letters + apostrophes)
    tokens = re.findall(r"[A-Za-z']+", txt)
    stems  = [stemmer.stem(w) for w in tokens]
    return " ".join(stems)

print(token("You are so impressive in, Mathematics and Physics, disaapointment"))


you are so impress in mathemat and physic disaapoint


In [55]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [57]:
df['text'].apply(lambda x : token(x))

0       beneath still water there' a strong undertow t...
1       i wa danc when i wa twelv i wa danc when i wa ...
2       thought about you just won't let me sleep at n...
3       open heart empti space dusti road to distant p...
4       hi smile like a million rose bath in rock 'n' ...
                              ...                        
4995    did you think you knew the game did you think ...
4996    well i gave you love you know it so when did y...
4997    just becaus in a moment of mad i took advantag...
4998    time will come when we know what happen here c...
4999    when did you know that noth wa right were you ...
Name: text, Length: 5000, dtype: object

In [58]:
tfid = TfidfVectorizer(analyzer ='word', stop_words = "english")

In [59]:
df['song']

0                   Beneath Still Waters
1                          Cosmic Dancer
2                              Nightmare
3                               Run Wild
4                                    Jfk
                      ...               
4995         Ra Ra For Red Rocking Horse
4996              Thinking Bout Somethin
4997    One Chain (Don't Make No Prison)
4998                 Anything Can Happen
4999                        Empty Handed
Name: song, Length: 5000, dtype: object

In [60]:
matrix = tfid.fit_transform(df['text'])

In [61]:
similar = cosine_similarity(matrix)

In [62]:
similar[1]

array([0.04975368, 1.        , 0.00727589, ..., 0.01148124, 0.0101542 ,
       0.02127512])

Recommender Function

In [63]:
from difflib import get_close_matches

def recommender_exact(song_name):
    """Your existing exact-match function (raises if not found)."""
    target = song_name.strip().lower()
    mask   = df['song'].str.strip().str.lower() == target
    if not mask.any():
        raise ValueError(f"Song '{song_name}' not found")
    idx = mask.idxmax()
    scored = sorted(enumerate(similar[idx]), key=lambda x: x[1], reverse=True)
    return [ df.iloc[s_id].song for s_id, _ in scored[1:6] ]

def recommender_song_lookup(query):
    """
    1) Try an exact match first.
    2) If that fails, look for titles containing the query (case-insensitive).
    3) If still nothing, fall back to fuzzy matching.
    """
    try:
        return recommender_exact(query)
    except ValueError:
        # 1) substring matches
        substr_mask = df['song'].str.lower().str.contains(query.lower(), regex=False)
        candidates  = df.loc[substr_mask, 'song'].tolist()
        if candidates:
            chosen = candidates[0]   # or let user pick among candidates
            print(f"No exact match – using '{chosen}'")
            return recommender_exact(chosen)

        # 2) no substring match → fuzzy match
        titles = df['song'].tolist()
        closest = get_close_matches(query, titles, n=1, cutoff=0.5)
        if closest:
            chosen = closest[0]
            print(f"No substring match – best fuzzy match is '{chosen}'")
            return recommender_exact(chosen)

        # 3) give up
        print(f"No match at all for '{query}'.")
        return []

# Example:
print(recommender_song_lookup("In your eyes"))


['Dream On', 'Dream Dancing', 'You Send Me', 'Dream A Lie', 'A Dream For Us']


In [64]:
import pickle

In [66]:
pickle.dump(similar,open("similarity", "wb"))

In [67]:
pickle.dump(df, open("df", "wb"))